# Working with riverine flooding data

In this tutorial an introduction to working with the World Resources Institute (WRI) Aqueduct flooding layers will be provided.

Why do water risks matter?

- An urgent global challenge. 
- Many public health crises are driven by water, including floods, droughts and water-borne diseases. 
- Climate change is worsening the problem by intensifying floods and drought, shifting precipitation patterns, altering water supplies and accelerating glacial melt and sea level rise. 
- Clean water supplies are vital for human health, industry, agriculture and energy production, making water risks a major humanitarian threat. 
- Identifying, understanding and responding to these risks requires transparent, publicly available data.


https://www.wri.org/

There are a number of dimensions we will explore with this data:

- Hazard types
- Climate scenarios
- Models
- Projection years
- Return periods




## Exercise 1.1

Navigate to the WRI Water Risk Atlas and familiarize yourself with the layers provided. 

Explore the key dimensions to water risk. 


# Downloading raw WRI Aqueduct data layers

We want to download the global data for us to explore on our own machine. 

Visit the WRI Aqueduct download page:

http://wri-projects.s3.amazonaws.com/AqueductFloodTool/download/v2/index.html
    
Download a .tif riverine flood layer for the MIROC model, for the RCP8.5 scenario projected to 2080, and a return period of 1-in-1000 years (thus, an annual probability of occurrence of 0.01%). 

Place the layer in `global_assessment/data/raw/flood_hazard`.

    

# Processing flood layers for each country

Now we are going to begin by processing one flooding layer for an example country.  

To do this, we are going to utilize `rasterio` and the `mask` function. 

Whereas in previous classes we loaded in raster data and exported our desired data from the global layer, today we will clip the raster layer for each country.

For example, the `mask` function is a bit like a "cookie cutter". For example, areas outside of a polygon boundary (e.g., country boundary) are excluded, enabling the internal areas to be exported.  

See more details here:

https://rasterio.readthedocs.io/en/stable/api/rasterio.mask.html

*Creates a masked or filled array using input shapes. Pixels are masked or set to nodata outside the input shapes, unless invert is True.*

Let us begin by loading in our desired packages, and getting our country lookup table data. We will focus only on Rwanda.

In [ ]:
# Example
import os
import json
import rasterio
from rasterio.mask import mask
import pandas
import geopandas 

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

for idx, country in countries.iterrows():

    if not country['iso3'] == 'RWA': 
        continue   

    print(country)   

This should be fairly familiar ground for you now. 

In which case, let us specify the file we just downloaded, which should be `inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif`.

We can load the raster layer, specify the no data value and define the crs. 


In [ ]:
# Example
for idx, country in countries.iterrows():
    
    if not country['iso3'] == 'RWA': 
        continue   
        
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
    path_hazard = os.path.join('..', 'data','raw','flood_hazard', filename)
    hazard = rasterio.open(path_hazard, 'r+')
    hazard.nodata = 255                       #set the no data value
    hazard.crs.from_epsg(4326)                #set the crs
    
    print(hazard)

Now we can load in country boundary for Rwanda, as follows.

Think of this boundary as the "cookie cutter" which we will apply to the global data layer. 


In [ ]:
# Example
path_country = os.path.join('..', 'data', 'processed', 'RWA', 'national_outline.shp')
country = geopandas.read_file(path_country)
country

Now we need to convert out country geometry into a json, which we weill feed to the mask function. 



In [ ]:
# Example
geo = geopandas.GeoDataFrame({'geometry': country.geometry})
coords = [json.loads(geo.to_json())['features'][0]['geometry']]

out_img, out_transform = mask(hazard, coords, crop=True)
out_img, out_transform


So the `out_img` is our data in numpy array form. And the `out_transform` is our affine transformation. 

Finally, we need to update the meta data for the new file we are about to write. 

In [ ]:
# Example
out_meta = hazard.meta.copy()
out_meta.update({"driver": "GTiff",
                "height": out_img.shape[1],
                "width": out_img.shape[2],
                "transform": out_transform,
                "crs": 'epsg:4326'})


So now we can specify where we want to export the data to.

In [ ]:
# Example
folder = os.path.join('..', 'data', 'processed', 'RWA', 'hazards', 'inunriver')
if not os.path.exists(folder):
    os.makedirs(folder)
path_out = os.path.join(folder, filename)

And finally write our file. So we pass to the `rasterio` write function the `out_img` data, along with the file path and the updated meta data. 

In [ ]:
# Example
with rasterio.open(path_out, "w", **out_meta) as dest:
        dest.write(out_img)

print('Processing complete.')

# Exercise 1.2

Can you assemble the full loop, to process the data for Rwanda? 

# Exercise 1.3

And now can you process the data for Granda?

Now we have our .tif file written out for a single country. We want to transform this data first to a shapefile. 

In the following example, you can use this code "out of the box" in order to convert your file into a shapefile. 

In [ ]:
for idx, country in countries.iterrows():
    
    if not country['iso3'] == 'RWA': # if the current country iso3 does not match RWA...
        continue                     # continue in the loop to the next country 
        
    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.shp'
    path_out = os.path.join(folder, filename)

    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
    path_in = os.path.join(folder, filename)

    with rasterio.open(path_in) as src:

        affine = src.transform
        array = src.read(1)

        output = []

        for vec in rasterio.features.shapes(array):

            if vec[1] > 0 and not vec[1] == 255:

                coordinates = [i for i in vec[0]['coordinates'][0]]

                coords = []

                for i in coordinates:

                    x = i[0]
                    y = i[1]

                    x2, y2 = src.transform * (x, y)

                    coords.append((x2, y2))

                output.append({
                    'type': vec[0]['type'],
                    'geometry': {
                        'type': 'Polygon',
                        'coordinates': [coords],
                    },
                    'properties': {
                        'value': vec[1],
                    }
                })

    output = geopandas.GeoDataFrame.from_features(output, crs='epsg:4326')
    output.to_file(path_out, driver='ESRI Shapefile')


Now you have your shapefile data layer in place, we can begin to plot that code.  

We will need our plotting packages.

In [ ]:
# Example
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx

First, let's set up our plotting interface.

In [ ]:
# Example
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

Set the path for our shapefile data.

In [ ]:
# Example
folder = os.path.join('..', 'data', 'processed', 'RWA', 'hazards', 'inunriver')
filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.shp'
path_in = os.path.join(folder, filename)
path_in

Load in our shapefile data.

In [ ]:
# Example
hazard = geopandas.read_file(path_in, crs='epsg:3857')
hazard = hazard.to_crs(4326)
hazard.plot(color='blue', linewidth=1.5, alpha=.7, legend=True, edgecolor=None, ax=ax)

We will want to add a `contextily` basemap.

In [ ]:
# Example
cx.add_basemap(ax, crs='epsg:4326')

And create a title for our plot.

In [ ]:
# Example
hazard_type = filename.split('_')[0]
scenario = filename.split('_')[1]
model = filename.split('_')[2]
year = filename.split('_')[3]
return_period = filename.split('_')[4]

return_period = return_period.replace('.shp', '')

main_title = 'Projected River Flooding: {}, {}, {}, {}, {}, {}'.format('RWA', hazard_type, scenario, model, year, return_period)
plt.suptitle(main_title, fontsize=14, wrap=True)

In [ ]:
Finally, we can write out our figure:

In [ ]:
# Example
path_out = os.path.join('..', 'data', 'processed', 'RWA', 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.png')

plt.savefig(path_out)
plt.close()

print('Figure processing complete')

# Exercise 1.4

Try generate a map for Grenada.

Convert the flood layer color to black. 